# ***Clustering sobre Noticias de "El Tiempo" (Colombia)***

## ***Objetivo***

Identificar, analizar y resumir los temas predominantes en las noticias de "El Tiempo", con un enfoque en detectar patrones emergentes. A lo largo del ejercicio, se buscará la identificación de temas en busca de insights estratégicos sobre el interés público y los temas de impacto en Colombia.



## ***Pasos Detallados***

### ***1. Obtención de URLs de Noticias***
- **Uso de Web Scraping**: Recolecta URLs no solo de la sección de noticias recientes, sino de múltiples secciones como política, economía, deportes, y cultura.


### ***2. Extracción de resumenes del Texto de la Noticia***

- **Procesamiento Previo de Texto**: Realiza limpieza del texto (remoción de ruido, normalización, etc.).
- **Análisis de Sentimiento Inicial**: Como paso previo al clustering, utiliza una herramienta de análisis de sentimiento para agregar contexto emocional a los textos, lo cual puede enriquecer la interpretación de los clusters.

### 3. ***Generación de Embeddings con [`Ollama`](https://ollama.com/) y [`langchain`](https://python.langchain.com/docs/)***
- **Exploración Comparativa de Embeddings**: Genera embeddings utilizando diferentes modelos de Ollama para comparar la representatividad y consistencia de cada modelo en la agrupación de temas.
.

### ***4. Aplicación de Métodos de Clustering***
- **Exploración de Diferentes Algoritmos**: Experimenta con técnicas de clustering jerárquico, K-Means, GaussMixture.

- **Selección del Número de Clusters Dinámicamente**: Usa métodos como la silueta y Davies-Bouldin para determinar el número óptimo de clusters y realiza pruebas cruzadas.

- **Interpretación Visual**: Genera gráficos de reducción de dimensionalidad con UMAP y t-SNE para visualizar los clusters y detectar posibles temas subyacentes.

### ***5. Análisis con un LLM Local para Caracterización y Resumen de Clusters***

- **Uso de LLM para Insight Profundo**: Utiliza Ollama en Colab para generar resúmenes detallados y descripciones de cada cluster, incluyendo:

  - Resúmenes temáticos generales.
  - Identificación de términos clave.
  - Resumen de sentimientos predominantes dentro de cada cluster.


### ***6. Discusión y Evaluación de Resultados***

- **Contraste de Resultados con Noticias Internacionales**: Veirificar la perticencia de los clusters formados y realizar limpieza post modelado.
- **Generación de Reporte Visual e Interactivo**: Crea un tablero visual en Colab o herramientas como Streamlit para presentar los resultados, que incluya insights automáticos generados por el LLM para cada cluster.
- **Discusión y Conclusiones Estratégicas**: Propón hipótesis sobre por qué ciertos temas ganan o pierden relevancia en el periodo de análisis.


In [1]:
import requests
import pandas as pd
from io import StringIO

## ***Obtener urls de noticias del Tiempo***

Una sesión HTTP se es creada mediante `requests.Session()`. Las sesiones permiten mantener ciertas configuraciones y cookies a través de múltiples solicitudes.

In [2]:
url_new = (
    """https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar"""
    """-el-titulo-en-la-proxima-carrera-en-japon-3385519"""
)

# Crear una sesión HTTP para mantener cookies y configuraciones en múltiples solicitudes
session = requests.Session()
# Peticion HTTP
response = session.get(url_new)
# Respuesta peticion
print("Respuesta a la peticion url:", response.status_code)
print(f"{100 * '='}")
# Contenido de la pagina web
print(response.text)

Respuesta a la peticion url: 200
<!DOCTYPE html>
    <html lang="es">
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
                                    

                                            
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1">
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
<title>David Alonso sigue imparable en Moto3: ganó en Indonesia y podría asegurar el título en la próxima carrera, en Japón</title>
<link href="https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar-el-titulo-en-la-proxima-carrera-en-japon-3385519" rel="canonical">
<link rel="alternate" href="https://www.eltiempo.com/deportes/otros-deportes/david-alonso-imparable-en-moto3-gano-en-indonesia-y-podria-asegurar-el-titulo-en-la-proxima-carrera-en-japon-3385519" hreflang="es-co"/>
<meta name

### ***¿Qué es un Sitemap XML?***
Un sitemap es un archivo en formato XML que contiene una lista de URLs de un sitio web. Su propósito principal es ayudar a los motores de búsqueda a encontrar, rastrear e indexar todas las páginas importantes del sitio de manera más eficiente. Esto es especialmente útil para sitios grandes o aquellos que actualizan contenido con frecuencia.

La URL https://www.eltiempo.com/sitemap-articles-current.xml apunta al sitemap de El Tiempo, un periódico digital. En este archivo, podemos encontrar las URLs de artículos recientes, que ayudan a que los motores de búsqueda como Google puedan descubrir e indexar rápidamente el contenido nuevo o actualizado.

In [3]:
url = 'https://www.eltiempo.com/sitemap-articles-current.xml'
session = requests.Session()
response = session.get(url)
if response.raise_for_status:
    print(f"OK, with the page: {url}. Status Code: {response.status_code}\n")
    print(f"Text from Page:")
    print(response.text[: 1000])
else:
    print(f"Problem in the page: {url}")

OK, with the page: https://www.eltiempo.com/sitemap-articles-current.xml. Status Code: 200

Text from Page:
<?xml version="1.0" encoding="UTF-8"?>
<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9" xmlns:video="http://www.google.com/schemas/sitemap-video/1.1" xmlns:news="http://www.google.com/schemas/sitemap-news/0.9">
 <url>
  <loc>https://www.eltiempo.com/justicia/conflicto-y-narcotrafico/por-que-hay-un-radical-cambio-de-estrategia-en-las-fuerzas-militares-fuerzas-de-tarea-conjuntas-pasaran-a-las-divisiones-3396449</loc>
  <lastmod>2024-11-03T22:10:00-05:00</lastmod>
  <news:news>
   <news:publication>
    <news:name>ElTiempo.com</news:name>
    <news:language>es</news:language>
   </news:publication>
   <news:title>¿Por qué hay un radical cambio de estrategia en las Fuerzas Militares? Fuerzas de Tarea Conjuntas pasarán a las divisiones</news:title>
   <news:publication_date>2024-11-03T22:10:00-05:00</news:publication_date>
   <news:keywords>por, qué, hay, un, radical</news:

El siguiente código extrae URLs de un sitemap XML y las convierte en un DataFrame de pandas para facilitar su análisis.

- Espacios de nombres XML: Define los espacios de nombres para identificar etiquetas en el XML:

- ns es el espacio principal del sitemap.

- news y video son espacios adicionales para etiquetas de noticias y videos.
- Lectura del XML:
    - pd.read_xml() lee el XML, convirtiendo el contenido en un DataFrame.
    - StringIO(response.text) convierte el contenido de response.text en un objeto similar a un archivo, que pd.read_xml() necesita para procesar el XML.
    - xpath=".//ns:url" selecciona las etiquetas <url> en el XML.
    - namespaces=namespaces asegura que los espacios de nombres definidos sean reconocidos.

- El DataFrame df_urls_news_el_tiempo contiene las URLs y sus datos asociados (URL, fecha de modificación, prioridad, etc.), que se muestran como una tabla en el notebook.

In [4]:
namespaces = {
    "ns": "http://www.sitemaps.org/schemas/sitemap/0.9",
    "news": "http://www.google.com/schemas/sitemap-news/0.9",
    "video": "http://www.google.com/schemas/sitemap-video/1.1"
}

# Hacemos lectura del texto XML
df_urls_news_el_tiempo = pd.read_xml(StringIO(response.text), xpath=".//ns:url", namespaces=namespaces)
df_urls_news_el_tiempo

,loc,lastmod,news
0,https://www.eltiempo.com/justicia/conflicto-y-...,2024-11-03T22:10:00-05:00,\n
1,https://www.eltiempo.com/economia/sector-finan...,2024-11-03T22:10:00-05:00,\n
2,https://www.eltiempo.com/mundo/eeuu-y-canada/l...,2024-11-03T22:10:00-05:00,\n
3,https://www.eltiempo.com/bogota/el-museo-ocult...,2024-11-03T22:10:00-05:00,\n
4,https://www.eltiempo.com/cultura/gente/citrato...,2024-11-03T22:03:46-05:00,\n
...,...,...,...
723,https://www.eltiempo.com/deportes/futbol-inter...,2024-11-01T00:00:00-05:00,\n
724,https://www.eltiempo.com/cultura/gente/logina-...,2024-11-01T00:00:00-05:00,\n
725,https://www.eltiempo.com/cultura/gente/cinco-s...,2024-11-01T00:00:00-05:00,\n
726,https://www.eltiempo.com/cultura/gente/las-per...,2024-11-01T00:00:00-05:00,\n


In [5]:
# Hacemos lectura del texto XML
df_title_date_keywords_news_el_tiempo = pd.read_xml(StringIO(response.text), xpath=".//news:news", namespaces=namespaces)
df_title_date_keywords_news_el_tiempo

,publication,title,publication_date,keywords
0,\n,¿Por qué hay un radical cambio de estrategia e...,2024-11-03T22:10:00-05:00,"por, qué, hay, un, radical"
1,\n,'El futuro de la región solo depende de nosotr...,2024-11-03T22:10:00-05:00,"el, futuro, de, la, región"
2,\n,¿Por qué los latinos tienen el potencial de in...,2024-11-03T22:10:00-05:00,"Estados Unidos, Donald trump, Kamala Harris, L..."
3,\n,Bogotá: el museo oculto del vestuario en minia...,2024-11-03T22:10:00-05:00,"el, museo, oculto, del, vestuario"
4,\n,"Citrato de magnesio: los dolores que alivia, s...",2024-11-03T22:03:46-05:00,"citrato, de, magnesio, los, dolores"
...,...,...,...,...
723,\n,Video | Juan Fernando Quintero celebra entre l...,2024-11-01T00:00:00-05:00,"Juan Fernando Quintero, Racing, Corinthians, S..."
724,\n,"Logina Salah, la primera participante del Miss...",2024-11-01T00:00:00-05:00,"Logina Salah, Primera, Participante, Miss Univ..."
725,\n,Cinco señales del cuerpo para advertir deficie...,2024-11-01T00:00:00-05:00,"Señales, Cuerpo, Advertencia, Deficiencia nutr..."
726,\n,¿Las personas de clima frío tienen menos deseo...,2024-11-01T00:00:00-05:00,"las, personas, de, clima, frío"


In [6]:
df_urls_news_el_tiempo = (
    df_urls_news_el_tiempo[["loc"]]
    .merge(
        df_title_date_keywords_news_el_tiempo.drop(columns=["publication"]),
        left_index=True,
        right_index=True        
    )
    .rename(columns={"loc": "url_page"})
)

df_urls_news_el_tiempo

,url_page,title,publication_date,keywords
0,https://www.eltiempo.com/justicia/conflicto-y-...,¿Por qué hay un radical cambio de estrategia e...,2024-11-03T22:10:00-05:00,"por, qué, hay, un, radical"
1,https://www.eltiempo.com/economia/sector-finan...,'El futuro de la región solo depende de nosotr...,2024-11-03T22:10:00-05:00,"el, futuro, de, la, región"
2,https://www.eltiempo.com/mundo/eeuu-y-canada/l...,¿Por qué los latinos tienen el potencial de in...,2024-11-03T22:10:00-05:00,"Estados Unidos, Donald trump, Kamala Harris, L..."
3,https://www.eltiempo.com/bogota/el-museo-ocult...,Bogotá: el museo oculto del vestuario en minia...,2024-11-03T22:10:00-05:00,"el, museo, oculto, del, vestuario"
4,https://www.eltiempo.com/cultura/gente/citrato...,"Citrato de magnesio: los dolores que alivia, s...",2024-11-03T22:03:46-05:00,"citrato, de, magnesio, los, dolores"
...,...,...,...,...
723,https://www.eltiempo.com/deportes/futbol-inter...,Video | Juan Fernando Quintero celebra entre l...,2024-11-01T00:00:00-05:00,"Juan Fernando Quintero, Racing, Corinthians, S..."
724,https://www.eltiempo.com/cultura/gente/logina-...,"Logina Salah, la primera participante del Miss...",2024-11-01T00:00:00-05:00,"Logina Salah, Primera, Participante, Miss Univ..."
725,https://www.eltiempo.com/cultura/gente/cinco-s...,Cinco señales del cuerpo para advertir deficie...,2024-11-01T00:00:00-05:00,"Señales, Cuerpo, Advertencia, Deficiencia nutr..."
726,https://www.eltiempo.com/cultura/gente/las-per...,¿Las personas de clima frío tienen menos deseo...,2024-11-01T00:00:00-05:00,"las, personas, de, clima, frío"


## ***Obtener el texto de las noticias***

In [7]:
from bs4 import BeautifulSoup

In [8]:
# Ejemplo de url a visitar:
df_urls_news_el_tiempo.iloc[0].loc["url_page"]

'https://www.eltiempo.com/justicia/conflicto-y-narcotrafico/por-que-hay-un-radical-cambio-de-estrategia-en-las-fuerzas-militares-fuerzas-de-tarea-conjuntas-pasaran-a-las-divisiones-3396449'

In [9]:
%%time
# Primeras 20 urls de noticias.
list_urls = df_urls_news_el_tiempo["url_page"].head(5).to_list()

for url in list_urls:
    print("Trabajando en la URL:", url)
    request = requests.get(url)
    soup = BeautifulSoup(request.content, "html.parser")
    
    # Verificar si el request es exitosa
    if request.raise_for_status:
        author_tag = soup.find("a", class_="c-detail__author__name").get_text()
        content = " ".join([tag.get_text() for tag in soup.find_all("div", class_="paragraph")])     
        print(f"Autor: {author_tag}")
        print(f"Contenido noticia: {content}")
        print()

Trabajando en la URL: https://www.eltiempo.com/justicia/conflicto-y-narcotrafico/por-que-hay-un-radical-cambio-de-estrategia-en-las-fuerzas-militares-fuerzas-de-tarea-conjuntas-pasaran-a-las-divisiones-3396449


Autor: Alicia Liliana Méndez
Contenido noticia: La Fuerza de Tarea Omega, conformada en 2003, ha sido una de las unidades castrenses insignia de las Fuerzas Militares por sus resultados contra la otrora guerrilla de las Farc y otros grupos delictivos. De hecho, nació como una necesidad operacional para enfrentar el poderío delincuencial que ejercía el ‘Mono Jojoy’, que entre otras cosas había convertido la zona rural de La Macarena (Meta) en un campo de concentración para mantener por años a centenares de secuestrados como Ingrid Betancourt o integrantes de la Fuerza Pública. (De seguro le interesa leer: Urgente: por amenazas de las disidencias 'Carlos Patiño', artistas como Arelys Henao cancelaron show en El Plateado) Los resultados operacionales de la Omega, que se concretaron en tener en una sola unidad a personal de Ejército, Armada y Fuerza Aérea (lo que facilitaba una reacción contundente), llevaron a la conformación de otras Fuerzas de Tarea Conjunta y con el paso de los años a 

Autor: ricardo ávila
Contenido noticia: Es descrito como el banco de inversión más grande de América Latina. Se trata de BTG Pactual, una entidad cuya casa matriz se encuentra en Brasil y cuenta con operaciones en Colombia, además de estar presente en varios países latinoamericanos, al igual que en los principales centros financieros mundiales. Sus más recientes datos financieros muestran una utilidad cercana a los 1.100 millones de dólares al cierre del primer semestre y activos bajo administración de más de 300.000 millones de dólares. Tal resultado equivale a una rentabilidad patrimonial superior al 22 por ciento, un nivel considerado récord. Desde mediados de la década pasada, su presidente ejecutivo es Roberto Sallouti, nacido en São Paulo y quien completa treinta años vinculado al conglomerado. La semana pasada, el ejecutivo estuvo en Bogotá y conversó con EL TIEMPO. Nos preciamos de conocer muy bien a la región y entendemos su volatilidad. Entonces contamos con un método de admi

Autor: JUAN DAVID NARANJO NAVARRO*
Contenido noticia: En 2015, cuando Donald Trump lanzó su primera candidatura presidencial en Estados Unidos, muchos lo tomaron como un chiste. Su retórica inicial, cargada de comentarios xenófobos, escandalizó a quienes lo escuchaban hablar de los migrantes latinos como “lo peor de sus países” y proponer construir un muro fronterizo con México. “Son violadores, traficantes, criminales”, dijo Trump, marcando su posición inicial hacia esta comunidad. (Vea también: Ola inédita de 'fake news' en elecciones de EE. UU.: así influyen la desinformación, las teorías falsas y de conspiración en votantes) Hoy, nueve años después, esos comentarios persisten, pero no sorprenden igual. Desde entonces, Trump no solo fue presidente (2017-2021) sino que su movimiento Make America Great Again (Hagamos a América Grande de Nuevo) transformó al Partido Republicano y, tras ser derrotado en 2020, intenta por tercera vez llegar a la Casa Blanca. La sorpresa, ahora, es que lo

Autor: ricardo rondón
Contenido noticia: Frente a los fogones de la Negra Hipólita, nodriza de Simón Bolívar, Carolina Trujillo enciende un cigarrillo, y el destello del cerillo, en la penumbra, se refleja en sus pupilas melancólicas. La escena no se produce en las tablas con uno de los tantos personajes que Trujillo, en su juventud, interpretó en el Teatro Popular de Bogotá (TPB), sino en 'Barcarola', un bar fantasma, náufrago en el tiempo, ubicado en los bajos de las emblemáticas Torres del Parque (del maestro Rogelio Salmona), donde la octogenaria y recordada actriz, escenógrafa y vestuarista, instaló hace ya casi 25 años su museo del traje en miniatura, desde la edad de piedra, y su evolución en la historia de la humanidad. La esclava Hipólita, que amamantó al Libertador, rodeada de trastos, canastos y chorotes, es uno de los más de 100 "monos", como Carolina llama a los modelos de su exposición, compañeros únicos en la novelesca vida de la virtuosa artista, que en su época dorada 

Autor: Noticias GDA
Contenido noticia: En los últimos tiempos, los suplementos alimenticios se convirtieron en protagonistas clave dentro del ámbito del bienestar personal, ya que son cada vez más personas las que toman conciencia de la importancia de cuidar la salud. En consecuencia, muchos ya optaron por integrarlos a la rutina diaria, ya sea para compensar la falta de nutrientes que no logran obtener con la alimentación regular, o bien como un apoyo adicional para reducir el riesgo de deficiencias. Entre los suplementos más destacados, el de magnesio ganó especial popularidad, debido a su papel esencial en múltiples funciones vitales. Según lo explica el sitio especializado MedlinePlus, el citrato de magnesio se emplea comúnmente como tratamiento a corto plazo para aliviar el estreñimiento. Esto debido a que este compuesto forma parte de una clase de medicamentos conocidos como laxantes salinos, que actúa reteniendo agua en las heces, lo cual facilita su expulsión. Este proceso no s

## ***Paralelizar la tarea de obtencion del contenido de las noticias***

In [10]:
session = requests.Session()

def get_content_news_from_url(url: str) -> dict:
    """
    Obtiene el contenido de una noticia desde una URL, extrayendo el autor y el cuerpo de la noticia.

    :param url: URL de la página de la noticia.
    :return: Diccionario con la URL, autor y contenido de la noticia.
    :raises ValueError: Si hay un problema con el status_code de la solicitud.
    """
    request = session.get(url)
    
    if request.raise_for_status:
        soup = BeautifulSoup(request.content, "html.parser")
        author_tag = soup.find("a", class_="c-detail__author__name").get_text()
        content = " ".join([tag.get_text() for tag in soup.find_all("div", class_="paragraph")])
        
        return {
            "url_page": url,
            "autor": author_tag,
            "news_content": content            
        }
    else:
        ValueError(
            f"Problemas con el status_code: {request.status_code}"
        )
        

get_content_news_from_url(df_urls_news_el_tiempo.iloc[0].loc["url_page"]) 

{'url_page': 'https://www.eltiempo.com/justicia/conflicto-y-narcotrafico/por-que-hay-un-radical-cambio-de-estrategia-en-las-fuerzas-militares-fuerzas-de-tarea-conjuntas-pasaran-a-las-divisiones-3396449',
 'autor': 'Alicia Liliana Méndez',
 'news_content': "La Fuerza de Tarea Omega, conformada en 2003, ha sido una de las unidades castrenses insignia de las Fuerzas Militares por sus resultados contra la otrora guerrilla de las Farc y otros grupos delictivos. De hecho, nació como una necesidad operacional para enfrentar el poderío delincuencial que ejercía el ‘Mono Jojoy’, que entre otras cosas había convertido la zona rural de La Macarena (Meta) en un campo de concentración para mantener por años a centenares de secuestrados como Ingrid Betancourt o integrantes de la Fuerza Pública. (De seguro le interesa leer:\xa0Urgente: por amenazas de las disidencias 'Carlos Patiño', artistas como Arelys Henao cancelaron show en El Plateado) Los resultados operacionales de la Omega, que se concretaro

In [11]:
import os
from typing import Dict
from concurrent.futures import ThreadPoolExecutor, as_completed

num_cores = os.cpu_count()
print(f"Número de cores disponibles: {num_cores}")

# Es recomendado dejar un core libre
# num_cores = num_cores - 1


def parallel_process_urls(urls: list[str]) -> list[Dict]:
    """
    Procesa múltiples URLs en paralelo utilizando ThreadPoolExecutor,
    dejando 2 cores libres.
    
    :param urls: Lista de URLs a procesar.
    :return: Lista de diccionarios con la información extraída de cada URL.
    """
    results = []
    
    with ThreadPoolExecutor(max_workers=num_cores) as executor:
        futures = [executor.submit(get_content_news_from_url, url) for url in urls]
        
        for future in as_completed(futures):
            try:
                results.append(future.result())
            except Exception as e:
                print(f"Error procesando la URL: {e}")
    
    return results

# Lista de URLs a procesar. Descomente la siguiente linea de codigo 
# para obtener todas las urls.
# Procesamos todas las URLs en paralelo
# urls_to_process = df_urls_news_el_tiempo["url_page"].tolist()

# urls de ejemplo, solo una muestra de 20 urls
urls_to_process = df_urls_news_el_tiempo["url_page"].sample(20).tolist()
# Procesamos todas las URLs en paralelo
processed_results = parallel_process_urls(urls_to_process)
# Resultados
print(processed_results)
pd.DataFrame(processed_results)

Número de cores disponibles: 16


[{'url_page': 'https://www.eltiempo.com/mundo/eeuu-y-canada/se-quejo-porque-un-hombre-no-pago-su-cafe-en-la-primera-cita-y-desato-este-debate-en-redes-3396083', 'autor': 'Facundo Segarra', 'news_content': 'El hecho de tener una primera cita con alguien no es para nada sencillo. Al no conocer aún a la otra persona, pueden surgir muchas situaciones incómodas para cualquiera de los dos, como por ejemplo tomar la decisión de no invitar la comida, lo que ocurrió con un hombre que se volvió viral. En Estados Unidos y buena parte del mundo es un debate existente, y se reforzó con la experiencia que compartió una mujer en sus redes. Una chica llamada Khristina se quejó en sus redes sociales de que un hombre con el que salió a una primera cita no quiso pagar su café, y esto desató un interminable debate entre los usuarios: algunos se mostraron de acuerdo con el accionar del joven, y otros defendieron a la muchacha. Después de que el hombre tomó esa postura, Khristina decidió rechazarle una segu

,url_page,autor,news_content
0,https://www.eltiempo.com/mundo/eeuu-y-canada/s...,Facundo Segarra,El hecho de tener una primera cita con alguien...
1,https://www.eltiempo.com/bogota/video-secretar...,Sergio Andrés Gamboa Mendivelso,Un hecho de un posible delito ambiental cometi...
2,https://www.eltiempo.com/politica/proceso-de-p...,Camilo A. Castillo,"El pasado 30 de octubre, el Consejo de Segurid..."
3,https://www.eltiempo.com/economia/sectores/bol...,Lina Quiroga Rubio,"La Bolsa Mercantil de Colombia, en su condició..."
4,https://www.eltiempo.com/salud/por-que-hay-gen...,redacción salud,El fenómeno de desmayarse al ver sangre es más...
5,https://www.eltiempo.com/economia/empresas/vol...,Laura Lesmes Díaz,Después de cuatro Latam Airlines logró regresa...
6,https://www.eltiempo.com/mundo/eeuu-y-canada/c...,Donato Del Blanco,La creencia de que el ciclo lunar puede influi...
7,https://www.eltiempo.com/tecnosfera/la-onu-ale...,Agencia EFE,La enorme cantidad de imágenes de abuso sexual...
8,https://www.eltiempo.com/politica/gobierno/pre...,Juan Sebastián Lombo Delgado,La dana que ha afectado a varias regiones de E...
9,https://www.eltiempo.com/colombia/otras-ciudad...,Laura usma,"Sobre la madrugada de este domingo, un desliza..."


In [12]:
df_news_el_tiempo = df_urls_news_el_tiempo.merge(
    pd.DataFrame(processed_results),
    on=["url_page"]    
)

display(df_news_el_tiempo)
# Descomente las siguientes lineas de codigo se desea guarda los resultados
# en un archivo parquet
# df_news_el_tiempo.to_parquet("data/df_noticias_el_tiempo.parquet")
# pd.read_parquet("data/df_noticias_el_tiempo.parquet")

,url_page,title,publication_date,keywords,autor,news_content
0,https://www.eltiempo.com/colombia/otras-ciudad...,Deslizamiento de tierra tiene sin agua a más d...,2024-11-03T20:25:39-05:00,"deslizamiento, tiene, sin, agua, a",Laura usma,"Sobre la madrugada de este domingo, un desliza..."
1,https://www.eltiempo.com/colombia/barranquilla...,Investigan panfleto con supuesto ‘plan pistola...,2024-11-03T12:23:45-05:00,"Soledad, Atlántico, Panfleto",Deivis Jhoan López Ortega,Las autoridades avanzan en las investigaciones...
2,https://www.eltiempo.com/justicia/investigacio...,Procuraduría profirió pliego de cargos al exre...,2024-11-03T09:29:52-05:00,"procuraduría, profirió, pliego, de, cargos",Alicia Liliana Méndez,La Procuraduría General informó que profirió p...
3,https://www.eltiempo.com/economia/sectores/bol...,Bolsa Mercantil anunció que hay más gas dispon...,2024-11-03T07:20:54-05:00,"bolsa, mercantil, anunció, que, hay",Lina Quiroga Rubio,"La Bolsa Mercantil de Colombia, en su condició..."
4,https://www.eltiempo.com/tecnosfera/la-onu-ale...,La ONU alerta sobre la avalancha de imágenes d...,2024-11-03T04:46:43-05:00,"Inteligencia artificial, explotación infantil,...",Agencia EFE,La enorme cantidad de imágenes de abuso sexual...
5,https://www.eltiempo.com/tecnosfera/apps/estas...,Estas son las aplicaciones que más datos móvil...,2024-11-03T00:00:00-05:00,"Aplicaciones, Datos, Internet, Streaming, Videos",EUROPA PRESS,Ante un escenario en el que el uso de datos mó...
6,https://www.eltiempo.com/mundo/eeuu-y-canada/l...,Los habitantes de Illinois recibirán un pago d...,2024-11-02T08:00:00-05:00,"los, habitantes, de, illinois, recibirán",Facundo Segarra,"Muchas veces, para apoyar a los residentes en ..."
7,https://www.eltiempo.com/economia/finanzas-per...,"Lotería de Medellín, Santander y Risaralda: ve...",2024-11-02T00:00:00-05:00,"Lotería de Medellín, Lotería de Santander, Lot...",Laura Daniela Alarcón Vargas,Todos los viernes en las noches se juegan la L...
8,https://www.eltiempo.com/salud/por-que-hay-gen...,¿Por qué hay gente que se desmaya cuando ve sa...,2024-11-02T00:00:00-05:00,"por, qué, hay, gente, que",redacción salud,El fenómeno de desmayarse al ver sangre es más...
9,https://www.eltiempo.com/politica/gobierno/qui...,"¿Quién es César Manrique, el director de Funci...",2024-11-02T00:00:00-05:00,"quién, es, césar, manrique, el",Juan Sebastián Lombo Delgado,"Tres meses pasaron desde que César Manrique, d..."


## ***Usar un LLM local***

In [13]:
# Instalar Ollama (disponible para Windows, Linux y MacOS)
# Aca se debe poner la guia de instalacion en Colab (pendiente)
# 1. Instalemos Ollama en Colab --> !curl -fsSL https://ollama.com/install.sh | sh
# 2. Verificamos su instalacion --> !ollama --version
# 3. Instalamos una terminal en Colab para ejecutar Ollama
#       !pip install colab-xterm
#       %load_ext colabxterm
#       %xterm
# 4. Servir Ollama: run en terminal ---> ollama serve
# 5. Descargar modelo: !ollama pull qwen2.5:7b
# 6. Probar modelo: 
#       %xterm
#       !ollama run qwen2.5:7b

In [14]:
# !curl -fsSL https://ollama.com/install.sh | sh

In [15]:
# !ollama --version

In [16]:
## Installemos una terminal en Colab para ejecutar Ollama
#!pip install colab-xterm
#%load_ext colabxterm
#%xterm
##run in terminal ---> ollama serve

In [17]:
# !ollama pull qwen2.5:7b

In [18]:
# %xterm
# ollama run qwen2.5:7b

In [19]:
#!pip install langchain-ollama==0.2.0

In [20]:
# Generar un chat con un llm_ollama
from langchain_ollama import ChatOllama

llm_ollama = ChatOllama(
    model="qwen2.5:7b",
    temperature=0.1,
    num_predict=1024,
    keep_alive=0
)

llm_response = llm_ollama.invoke("¿Cuál es la segunda letra del alfabeto griego?")
print(llm_response)

content='La segunda letra del alfabeto griego es "Beta" (Β ηβ), que se escribe así en mayúscula y así en minúscula.' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-11-04T03:11:55.820447592Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4807185788, 'load_duration': 3889201521, 'prompt_eval_count': 43, 'prompt_eval_duration': 52174000, 'eval_count': 38, 'eval_duration': 772479000} id='run-96af5325-5455-4af2-be52-20633f90c573-0' usage_metadata={'input_tokens': 43, 'output_tokens': 38, 'total_tokens': 81}


In [21]:
%%time
llm_ollama.invoke(
    """Dame un lista de 20 personajes influyentes en la actualidad"""
    """mundial. No olvides incluir diferentes angulos de la vida. """
    """Solo debes darme la lista sin decir nada mas. Por ejemplo si """
    """decides incluir a Pepito Perez (suponiendo que es influyente """
    """actualmente) respuesta debe ser: Pepito Perez"""
    )

CPU times: user 31.8 ms, sys: 3.92 ms, total: 35.7 ms
Wall time: 4.3 s


AIMessage(content='Bill Gates\nElon Musk\nAngela Merkel\nJoe Biden\nXi Jinping\nBoris Johnson\nIndira Gandhi\nMalala Yousafzai\nGreta Thunberg\nOprah Winfrey\nJack Ma\nBernie Sanders\nTaylor Swift\n Cristiano Ronaldo\nSerena Williams\nSimone Biles\nPope Francis\nShakira\nMark Zuckerberg', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2024-11-04T03:12:00.121665299Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4293042668, 'load_duration': 2527192264, 'prompt_eval_count': 102, 'prompt_eval_duration': 86521000, 'eval_count': 78, 'eval_duration': 1625956000}, id='run-64358ec8-55d0-411b-b414-bd4c56abade2-0', usage_metadata={'input_tokens': 102, 'output_tokens': 78, 'total_tokens': 180})

In [22]:
# PromptTemplate y respuesta como string
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que crea excelentes biografias de """
            """personajes influyentes de la actualidad mundial y detalla su """
            """vida en diferentes aspectos. Responde usando solo JSON con """
            """llave el nombre del personaje y valor su biografia, """
            """Por ejemplo: dict(Pepito Perez="Pepito Peres ... (biografia))"""),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

print(prompt)

chain = prompt | llm_ollama | StrOutputParser()
chain.invoke(
    {
        "user_input": "Barack Obama"
    }
)

input_variables=['user_input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Eres una util AI bot que crea excelentes biografias de personajes influyentes de la actualidad mundial y detalla su vida en diferentes aspectos. Responde usando solo JSON con llave el nombre del personaje y valor su biografia, Por ejemplo: dict(Pepito Perez="Pepito Peres ... (biografia))'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})]


'{\n  "Barack Obama": "Barack Obama es un político estadounidense que sirvió como el 44º presidente de los Estados Unidos desde 2009 hasta 2017. Nació en Honolulu, Hawái, el 4 de agosto de 1961, y su ascendencia incluye padres de diferentes orígenes: su madre, Ann Dunham, era una americana de origen hawaiano y keniata, mientras que su padre, Barack Obama Sr., era un keniano. Obama asistió a la Universidad Columbia y a la Universidad Harvard, donde se graduó con un doctorado en Derecho y Ciencias Políticas.\\n\\nAntes de ser presidente, Obama fue senador por Illinois desde 2005 hasta 2008. Fue elegido presidente en las elecciones presidenciales de 2008, convirtiéndose en el primer afroamericano en ocupar la Casa Blanca. Durante su mandato, Obama implementó reformas significativas en áreas como la salud, la educación y la economía, y también se destacó por sus esfuerzos internacionales para promover la paz y la cooperación global.\\n\\nDespués de dejar el cargo presidencial, Obama ha seg

In [23]:
# Usamos JSON-format para parsear la respuesta del LLM
from langchain_core.output_parsers import JsonOutputParser

json_llm = ChatOllama(
    model="qwen2.5:7b",
    temperature=0.1,
    num_predict=-1,
    format="json",
    keep_alive=0
)

prompt = ChatPromptTemplate(
    [
        SystemMessagePromptTemplate.from_template(
            """Eres una util AI bot que crea excelentes biografias de """
            """personajes influyentes de la actualidad mundial y detalla su """
            """vida ampliamente en diferentes aspectos. No debes escatimar """
            """en la longitud del texto de resumen. Responde usando solo JSON """
            """format con llave el nombre del personaje y valor su biografia"""
            """Por ejemplo: dict(Pepito Perez="Pepito Perez ... (biografia)","""
            """ Benito Camelas="Benito Camelas ... (biografia)"). Debes """
            """seguir al pie de la letra el formato"""
            ),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

chain = prompt | json_llm | JsonOutputParser()
llm_answer = chain.invoke({"user_input": "Dame la biografia de 2 personajes"})
print(llm_answer, type(llm_answer))

{'Bill Gates': "Bill Gates, nacido el 28 de octubre de 1955 en Seattle, Washington, es un empresario, filántropo y escritor estadounidense. Es co-fundador de Microsoft Corporation junto con Paul Allen, donde ocupó la posición de presidente hasta 1981 y luego como director ejecutivo hasta 2000. Gates se retiró oficialmente de Microsoft en 2008 pero sigue siendo un miembro del consejo de administración. En 2000, junto con su esposa Melinda, fundó la Fundación Bill & Melinda Gates, una organización sin fines de lucro dedicada a mejorar las condiciones de vida en el mundo, especialmente en áreas como la salud y la educación. La fundación se ha convertido en una de las más grandes del mundo en términos de donaciones. Además de su trabajo filantrópico, Gates es autor de varios libros sobre tecnología y gestión empresarial, incluyendo 'Business @ the Speed of Thought' y 'The Road Ahead'. Su interés por la innovación tecnológica y el bienestar global ha hecho que sea una figura influyente en e

### ***Embeddings***

In [24]:
# !ollama pull mxbai-embed-large:latest

In [25]:
%%time
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)

question = ["¿Cuál es la segunda letra del alfabeto griego?"]
embed_question = ollama_embeddings.embed_documents(question)
print("Embeddings:")
pd.DataFrame(embed_question, index=question)

Embeddings:
CPU times: user 127 ms, sys: 602 μs, total: 128 ms
Wall time: 1.91 s


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
¿Cuál es la segunda letra del alfabeto griego?,-0.04817,0.015093,0.041272,0.001454,-0.037321,-0.011506,0.014483,0.003339,0.024329,-0.007726,...,0.045982,-0.015122,0.012864,-0.031581,0.009468,0.03363,-0.001437,0.034952,-0.060254,-0.037693


In [26]:
# Ejemplo: (Relevancia - Distancia)
from sklearn.metrics.pairwise import cosine_similarity
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)

texts = [
    "Alpha es la primera letra del alfabeto griego",
    "A es la primera letra del alfabeto latino",
    "Beta es la segunda letra del alfabeto griego",
    "B es la segunda letra del alfabeto latino"
]
query = ["¿Cuál es la segunda letra del alfabeto griego?"]
embeds = ollama_embeddings.embed_documents(texts)
embed_query = ollama_embeddings.embed_documents(query)
print(f"Long Ollama Embeddings: {len(embed_query[0])}")
print(f"Cosine Similarities: \n {cosine_similarity(embeds, embed_query)}")

pd.DataFrame(
    cosine_similarity(embeds, embed_query),
    index=texts,
    columns=query
).sort_values(by=query, ascending=False)

Long Ollama Embeddings: 1024
Cosine Similarities: 
 [[0.76798198]
 [0.77720013]
 [0.89850202]
 [0.8555794 ]]


,¿Cuál es la segunda letra del alfabeto griego?
Beta es la segunda letra del alfabeto griego,0.898502
B es la segunda letra del alfabeto latino,0.855579
A es la primera letra del alfabeto latino,0.777200
Alpha es la primera letra del alfabeto griego,0.767982


In [27]:
def get_summary_news(news: str, model_name: str = "qwen2.5:7b") -> str:

    llm_ollama = ChatOllama(
        model=model_name,
        temperature=0.1,
        num_predict=4096,
        num_ctx=16000,
        keep_alive=0
    )

    prompt_summarization = ChatPromptTemplate(
        [
            SystemMessagePromptTemplate.from_template(
                """Por favor, crea un resumen natural, claro y preciso de esta noticia. Asegúrate de incluir los 
                siguientes puntos de manera fluida::"""
                """1. Evento principal o tema central: Describe en una o dos frases el evento clave o el tema más """
                """importante de la noticia."""
                """2. Actores principales y sus reacciones: Menciona las partes involucradas y sus posturas o 
                reacciones significativas (gobiernos, empresas, expertos, etc.)."""
                """3. Consecuencias o implicaciones: Señala las implicaciones principales o cualquier posible """
                """consecuencia a corto o largo plazo derivada de la noticia."""
                """4. Contexto adicional relevante: Incluye cualquier información de contexto que sea relevante para """
                """comprender mejor el evento o las decisiones tomadas."""
                """El resumen debe ser natural y no exceder las cinco frases. Evita juicios o interpretaciones, 
                    enfocándote en capturar la esencia de la noticia sin perder detalles importantes."""
            ),
            HumanMessagePromptTemplate.from_template("El texto de la noticias es:\n{news}")
        ]
    )

    llm_chain = prompt_summarization | llm_ollama | StrOutputParser()

    return llm_chain.invoke({"news": news})

In [28]:
df_test = df_news_el_tiempo.sample(2)
df_test = df_test[["url_page", "news_content"]]
df_test["summary"] = df_test["news_content"].apply(get_summary_news)
df_test

,url_page,news_content,summary
9,https://www.eltiempo.com/politica/gobierno/qui...,"Tres meses pasaron desde que César Manrique, d...","César Manrique, director de la Función Pública..."
2,https://www.eltiempo.com/justicia/investigacio...,La Procuraduría General informó que profirió p...,La Procuraduría General presentó cargos contra...


In [29]:
for _, row in df_test.iterrows():
    print(f"URL: {row['url_page']}")
    print(f"Noticia: {row['news_content']}")
    print(f"Resumen: {row['summary']}\n")

URL: https://www.eltiempo.com/politica/gobierno/quien-es-cesar-manrique-el-director-de-funcion-publica-mencionado-en-el-escandalo-de-la-ungrd-que-renuncio-a-su-cargo-3396006
Noticia: Tres meses pasaron desde que César Manrique, director de la Función Pública, fue mencionado como supuesto participe del escándalo de la UNGRD y su salida del gobierno. Este viernes se conoció que daba un paso al costado. A pesar de la polémica que lo rodeó, aseveró que su salida fue por motivos de salud. (Puede ver: César Manrique, director de la Función Pública vinculado al escándalo de la UNGRD, renunció a su cargo en el gobierno de Gustavo Petro) La carrera pública de Manrique ha sido, en su mayoría, de la mano del presidente Gustavo Petro. Al consultar a miembros del Gobierno y del Pacto Histórico, estos lo identificaron como cercano al mandatario, “del combo de la izquierda”. Ha sido funcionario tanto en el gobierno distrital de Gustavo Petro en Bogotá como ahora de Presidente. En ambos casos ocupó im

In [30]:
# Embedding de las noticias.
from langchain_ollama import OllamaEmbeddings

ollama_embeddings = OllamaEmbeddings(
    model="mxbai-embed-large:latest",
)

embed_news = ollama_embeddings.embed_documents(df_test["summary"].to_list())
print("Embeddings:")
pd.DataFrame(embed_news, index=df_test["url_page"])

Embeddings:


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
url_page,,,,,,,,,,,,,,,,,,,,,
https://www.eltiempo.com/politica/gobierno/quien-es-cesar-manrique-el-director-de-funcion-publica-mencionado-en-el-escandalo-de-la-ungrd-que-renuncio-a-su-cargo-3396006,-0.021806,-0.022521,0.004087,0.020791,0.026071,-0.015417,-0.069267,-0.013593,0.029151,0.039393,...,0.048351,0.015442,-0.010847,-0.013942,0.018190,0.011914,-0.021167,0.020305,0.004352,0.015404
https://www.eltiempo.com/justicia/investigacion/procuraduria-profirio-pliego-de-cargos-al-exrector-de-la-universidad-militar-luis-fernando-puentes-torres-esto-se-sabe-3396377,-0.016873,-0.017927,0.006363,0.008675,-0.002774,0.004136,-0.056931,0.030544,0.022309,0.046123,...,0.037297,0.013146,-0.015416,0.004361,0.018977,0.010150,-0.026987,-0.021462,-0.009447,-0.000300
